### Deploy Web App on Azure Container Services (AKS)
In this notebook, we will set up an Azure Container Service which will be managed by Kubernetes. We will then take the Docker image we created earlier that contains our app and deploy it to the AKS cluster. Then, we will check everything is working by sending an image to it and getting it scored.
    
The process is split into the following steps:
* [Define our resource names](#section1)
* [Login to Azure](#section2)
* [Create resource group and create AKS](#section3)
* [Connect to AKS](#section4)
* [Deploy our app](#section5)

This guide assumes is designed to be run on linux and requires that the Azure CLI is installed.

In [1]:
import os
import json
from testing_utilities import write_json_to_file
from dotenv import set_key, get_key

<a id='section1'></a>
## Setup
Below are the various name definitions for the resources needed to setup AKS.

In [2]:
set_key('.env', 'selected_subscription', 'Team Danielle Internal')
set_key('.env', 'resource_group', 'msaksrg')
set_key('.env', 'aks_name', 'msaks')
set_key('.env', 'location', 'eastus')

(True, 'location', 'eastus')

In [3]:
# %%writefile --append .env
# # This cell is tagged `parameters`
# # Please modify the values below as you see fit

# # If you have multiple subscriptions select the subscription you want to use 
# selected_subscription = "Team Danielle Internal"

# # Resource group, name and location for AKS cluster.
# resource_group = "mabouaks" 
# aks_name = "mabouaks"
# location = "eastus"

In [4]:
image_name = get_key('.env', 'docker_login') + '/' +get_key('.env', 'image_repo') 

<a id='section2'></a>
## Azure account login
If you are not already logged in to an Azure account, the command below will initiate a login. It will pop up a browser where you can select an Azure account.

In [5]:
%%bash
list=`az account list -o table`
if [ "$list" == '[]' ] || [ "$list" == '' ]; then 
  az login -o table
else
  az account list -o table 
fi

Name                                            CloudName    SubscriptionId                        State    IsDefault
----------------------------------------------  -----------  ------------------------------------  -------  -----------
Boston DS Dev                                   AzureCloud   0ca618d2-22a8-413a-96d0-0f1b531129c3  Enabled  False
Azure Internal - London                         AzureCloud   1ba81249-8edd-4619-a486-3d28a2176aad  Enabled  False
Team Danielle Internal                          AzureCloud   edf507a2-6235-46c5-b560-fd463ba2e771  Enabled  True
Visual Studio Enterprise                        AzureCloud   fb11e9eb-22e1-4347-8d0a-84ef60157664  Enabled  False
Azure Stack Diagnostics CI and Production VaaS  AzureCloud   a8183b2d-7a4c-45e9-8736-dac11b84ff14  Enabled  False
Core-ES-BLD                                     AzureCloud   54e18c35-3863-4a17-8e52-b5aa1e65847e  Enabled  False
PhillyExt                                       AzureCloud   a20c82c7-4497-4d44

In [7]:
!az account set --subscription "{get_key('.env', 'selected_subscription')}"

In [8]:
!az account show

{
  "environmentName": "AzureCloud",
  "id": "edf507a2-6235-46c5-b560-fd463ba2e771",
  "isDefault": true,
  "name": "Team Danielle Internal",
  "state": "Enabled",
  "tenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
  "user": {
    "name": "masalvar@microsoft.com",
    "type": "user"
  }
}


In [9]:
!az provider register -n Microsoft.ContainerService

Registering is still on-going. You can monitor using 'az provider show -n Microsoft.ContainerService'


In [10]:
!az provider show -n Microsoft.ContainerService

{
  "authorization": {
    "applicationId": "7319c514-987d-4e9b-ac3d-d38c4f427f4c",
    "managedByRoleDefinitionId": "8e3af657-a8ff-443c-a75c-2fe8c4bcb635",
    "roleDefinitionId": "1b4a0c7f-2217-416f-acfa-cf73452fdc1c"
  },
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/providers/Microsoft.ContainerService",
  "namespace": "Microsoft.ContainerService",
  "registrationState": "Registered",
  "resourceTypes": [
    {
      "aliases": null,
      "apiVersions": [
        "2017-07-01",
        "2017-01-31",
        "2016-09-30",
        "2016-03-30"
      ],
      "capabilities": "None",
      "locations": [
        "Japan East",
        "Central US",
        "East US 2",
        "Japan West",
        "East Asia",
        "South Central US",
        "Australia East",
        "Australia Southeast",
        "Brazil South",
        "Southeast Asia",
        "West US",
        "North Central US",
        "West Europe",
        "North Europe",
        "East US",
        "UK West"

<a id='section3'></a>
## Create resource group and create AKS

### Create resource group
Azure encourages the use of groups to organise all the Azure components you deploy. That way it is easier to find them but also we can deleted a number of resources simply by deleting the group.

In [22]:
!az group create --name {get_key('.env', 'resource_group')} \
                 --location {get_key('.env', 'location')}

{
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/msaksrg",
  "location": "eastus",
  "managedBy": null,
  "name": "msaksrg",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null
}


Below, we create the AKS cluster in the resource group we created earlier. This can take up to 15 minutes.

In [23]:
!az aks create --resource-group {get_key('.env', 'resource_group')}  \
               --name {get_key('.env', 'aks_name')} \
               --node-count 1 \
               --generate-ssh-keys \
               -s Standard_NC6

SSH key files '/home/mat/.ssh/id_rsa' and '/home/mat/.ssh/id_rsa.pub' have been generated under ~/.ssh to allow SSH access to the VM. If using machines without permanent storage like Azure Cloud Shell without an attached file share, back up your keys to a safe location
{- Finished ..principal creation[##################################]  100.0000%
  "aadProfile": null,
  "addonProfiles": null,
  "agentPoolProfiles": [
    {
      "count": 1,
      "maxPods": 110,
      "name": "nodepool1",
      "osDiskSizeGb": null,
      "osType": "Linux",
      "storageProfile": "ManagedDisks",
      "vmSize": "Standard_NC6",
      "vnetSubnetId": null
    }
  ],
  "dnsPrefix": "msaks-msaksrg-edf507",
  "enableRbac": true,
  "fqdn": "msaks-msaksrg-edf507-be498e4a.hcp.eastus.azmk8s.io",
  "id": "/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/msaksrg/providers/Microsoft.ContainerService/managedClusters/msaks",
  "kubernetesVersion": "1.9.9",
  "linuxProfile": {
    "adminUsername":

### Install kubectl CLI

To connect to the Kubernetes cluster, we will use kubectl, the Kubernetes command-line client. To install, run the following:

In [24]:
!sudo az aks install-cli

Please ensure that /usr/local/bin is in your search PATH, so the `kubectl` command can be found.


<a id='section4'></a>
## Connect to AKS cluster

To configure kubectl to connect to the Kubernetes cluster, run the following command:

In [25]:
!az aks get-credentials --resource-group $resource_group --name $aks_name

Merged "msaks" as current context in /home/mat/.kube/config


Let's verify connection by listing the nodes.

In [27]:
!kubectl get nodes

NAME                       STATUS   ROLES   AGE    VERSION
aks-nodepool1-24684105-0   Ready    agent   3m5s   v1.9.9


Let's check the pods on our cluster.

In [28]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY   STATUS    RESTARTS   AGE
kube-system   heapster-6f4d8b48bc-bgvf5               2/2     Running   0          2m44s
kube-system   kube-dns-v20-7d874cb9b6-n9sg4           3/3     Running   0          2m45s
kube-system   kube-dns-v20-7d874cb9b6-q57jh           3/3     Running   0          2m44s
kube-system   kube-proxy-5qfds                        1/1     Running   0          2m42s
kube-system   kube-svc-redirect-2lpz9                 2/2     Running   0          2m41s
kube-system   kubernetes-dashboard-68f8cc4d8c-zdx2c   1/1     Running   0          2m44s
kube-system   tunnelfront-57555d5d8f-ps6vp            1/1     Running   0          2m42s


<a id='section5'></a>
## Deploy application

Below we define our Kubernetes manifest file for our service and load balancer. Note that we have to specify the volume mounts to the drivers that are located on the node.


In [30]:
app_template = {
  "apiVersion": "apps/v1beta1",
  "kind": "Deployment",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "replicas":1,
      "template":{
          "metadata":{
              "labels":{
                  "app":"azure-dl"
              }
          },
          "spec":{
              "containers":[
                  {
                      "name": "azure-dl",
                      "image": image_name,
                      "env":[
                          {
                              "name": "LD_LIBRARY_PATH",
                              "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"
                          }
                      ],
                      "ports":[
                          {
                              "containerPort":80,
                              "name":"model"
                          }
                      ],
                      "volumeMounts":[
                          {
                            "mountPath": "/usr/local/nvidia",
                            "name": "nvidia"
                          }
                      ],
                      "resources":{
                           "requests":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           },
                           "limits":{
                               "alpha.kubernetes.io/nvidia-gpu": 1
                           }
                       }  
                  }
              ],
              "volumes":[
                  {
                      "name": "nvidia",
                      "hostPath":{
                          "path":"/usr/local/nvidia"
                      },
                  },
              ]
          }
      }
  }
}

service_temp = {
  "apiVersion": "v1",
  "kind": "Service",
  "metadata": {
      "name": "azure-dl"
  },
  "spec":{
      "type": "LoadBalancer",
      "ports":[
          {
              "port":80
          }
      ],
      "selector":{
            "app":"azure-dl"
      }
   }
}

In [31]:
write_json_to_file(app_template, 'az-dl.json') # We write the service template to the json file

In [32]:
write_json_to_file(service_temp, 'az-dl.json', mode='a') # We add the loadbelanacer template to the json file

Let's check the manifest created.

In [33]:
!cat az-dl.json

{
    "apiVersion": "apps/v1beta1",
    "kind": "Deployment",
    "metadata": {
        "name": "azure-dl"
    },
    "spec": {
        "replicas": 1,
        "template": {
            "metadata": {
                "labels": {
                    "app": "azure-dl"
                }
            },
            "spec": {
                "containers": [
                    {
                        "env": [
                            {
                                "name": "LD_LIBRARY_PATH",
                                "value": "$LD_LIBRARY_PATH:/usr/local/nvidia/lib64:/opt/conda/envs/py3.6/lib"
                            }
                        ],
                        "image": "masalvar/pytorch-gpu",
                        "name": "azure-dl",
                        "ports": [
                            {
                                "containerPort": 80,
                                "name": "model"
                            }
                        ],
             

Next, we will use kubectl create command to deploy our application.

In [34]:
!kubectl create -f az-dl.json

deployment.apps/azure-dl created
service/azure-dl created


Let's check if the pod is deployed.

In [37]:
!kubectl get pods --all-namespaces

NAMESPACE     NAME                                    READY   STATUS    RESTARTS   AGE
default       azure-dl-75d565c95b-dh2bv               1/1     Running   0          8m10s
kube-system   heapster-6f4d8b48bc-bgvf5               2/2     Running   0          11m
kube-system   kube-dns-v20-7d874cb9b6-n9sg4           3/3     Running   0          11m
kube-system   kube-dns-v20-7d874cb9b6-q57jh           3/3     Running   0          11m
kube-system   kube-proxy-5qfds                        1/1     Running   0          11m
kube-system   kube-svc-redirect-2lpz9                 2/2     Running   0          11m
kube-system   kubernetes-dashboard-68f8cc4d8c-zdx2c   1/1     Running   0          11m
kube-system   tunnelfront-57555d5d8f-ps6vp            1/1     Running   0          11m


If anything goes wrong you can use the commands below to observe the events on the node as well as review the logs.

In [38]:
!kubectl get events

LAST SEEN   TYPE     REASON                    KIND         MESSAGE
13m         Normal   Starting                  Node         Starting kubelet.
12m         Normal   NodeHasSufficientDisk     Node         Node aks-nodepool1-24684105-0 status is now: NodeHasSufficientDisk
12m         Normal   NodeHasSufficientMemory   Node         Node aks-nodepool1-24684105-0 status is now: NodeHasSufficientMemory
12m         Normal   NodeHasNoDiskPressure     Node         Node aks-nodepool1-24684105-0 status is now: NodeHasNoDiskPressure
13m         Normal   NodeAllocatableEnforced   Node         Updated Node Allocatable limit across pods
11m         Normal   RegisteredNode            Node         Node aks-nodepool1-24684105-0 event: Registered Node aks-nodepool1-24684105-0 in Controller
10m         Normal   Starting                  Node         Starting kube-proxy.
8m16s       Normal   Scheduled                 Pod          Successfully assigned azure-dl-75d565c95b-dh2bv to aks-nodepool1-24684105-0

In [39]:
pod_json = !kubectl get pods -o json
pod_dict = json.loads(''.join(pod_json))
!kubectl logs {pod_dict['items'][0]['metadata']['name']}

2018-10-08 09:24:02,645 CRIT Supervisor running as root (no user in config file)
2018-10-08 09:24:02,646 INFO supervisord started with pid 1
2018-10-08 09:24:03,648 INFO spawned: 'program_exit' with pid 9
2018-10-08 09:24:03,649 INFO spawned: 'nginx' with pid 10
2018-10-08 09:24:03,651 INFO spawned: 'gunicorn' with pid 11
2018-10-08 09:24:04,679 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)
Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.torch/models/resnet152-b121ed2d.pth
100.0%2018-10-08 09:24:09,063 INFO success: nginx entered RUNNING state, process has stayed up for > than 5 seconds (startsecs)
{"timestamp": "2018-10-08T09:24:11.116435Z", "message": "Model loading time: 6937.52 ms", "host": "azure-dl-75d565c95b-dh2bv", "path": "/code/driver.py", "tags": [], "level": "INFO", "logger": "model_driver", "stack_info": null}
Initialising
{"timestamp": "2018-10-08T09:24:11.128540Z", "message": "

It can take a few minutes for the service to populate the EXTERNAL-IP field. This will be the IP you use to call the service. You can also specify an IP to use please see the AKS documentation for further details.

In [40]:
!kubectl get service azure-dl

NAME       TYPE           CLUSTER-IP   EXTERNAL-IP      PORT(S)        AGE
azure-dl   LoadBalancer   10.0.99.71   40.117.184.229   80:31432/TCP   8m33s


Next, we will [test our web application](05_TestWebApp.ipynb) deployed on AKS. 